# Project Architecture: Modular and Reusable Database Layer Design with FastAPI

## 1. Web Layer
This is the presentation layer that handles HTTP requests and responses, using FastAPI.

## 2. Service Layer
Contains business logic, acting as a mediator between the Web Layer and the Data & DAO Layer.

## 3. Data & DAO Layer
Handles database interactions, including configuration, initialization, base models, session management, and DAOs.



## Project Structure

In [ ]:
project_root/
│
├── app/
│   ├── __init__.py
│   ├── main.py                 # Entry point for the FastAPI application
│   │
│   ├── api/                    # Web layer with API routes
│   │   ├── __init__.py
│   │   └── v1/
│   │       ├── __init__.py
│   │       └── users.py        # API routes for User operations
│   │
│   ├── core/                   # Core settings and configuration
│   │   ├── __init__.py
│   │   └── config.py           # Configuration file
│   │
│   ├── db/                     # Database initialization and session management
│   │   ├── __init__.py
│   │   ├── base_model.py       # Base model definition
│   │   ├── init_db.py          # Database initialization script
│   │   └── session.py          # Session management
│   │
│   ├── models/                 # Database models
│   │   ├── __init__.py
│   │   └── user.py             # User model
│   │
│   ├── schemas/                # Pydantic models for request and response schemas
│   │   ├── __init__.py
│   │   └── user.py             # Schemas related to User
│   │
│   ├── services/               # Business logic (Service Layer)
│   │   ├── __init__.py
│   │   └── user_service.py     # User service for business logic
│   │
│   └── dao/                    # Data Access Objects (DAO Layer)
│       ├── __init__.py
│       └── user_dao.py         # DAO for User model
│
└── requirements.txt            # Dependencies


# Description

## app/main.py
The entry point for the FastAPI application.

## app/api/
Contains sub-folders for different versions of the API.  
Each version folder includes route handlers for entities like users.

## app/core/config.py
Configuration file to manage settings like the database URI.

## app/db/
Contains scripts for initializing the database and managing sessions.  
- `base_model.py`: Holds the abstract class for models.  
- `init_db.py`: Initializes database tables.  
- `session.py`: Manages DB sessions.

## app/models/
Holds ORM models, e.g., `user.py` for the User model.

## app/schemas/
Pydantic models for data validation and serialization.  
E.g., `user.py` for User-related schemas.

## app/services/
Business logic layer, containing service classes to handle more complex operations.

## app/dao/
Data access objects, encapsulating database interaction logic, making it easier to swap out for different storage mechanisms


## Configuration

In [ ]:
import os

class Config:
    SQLALCHEMY_DATABASE_URI = os.getenv('DATABASE_URI', 'sqlite:///./test.db')
    SQLALCHEMY_TRACK_MODIFICATIONS = False


## Database Initialization

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.orm import scoped_session, sessionmaker, declarative_base

engine = create_engine(Config.SQLALCHEMY_DATABASE_URI)
db_session = scoped_session(sessionmaker(autocommit=False, autoflush=False, bind=engine))

Base = declarative_base()

def init_db():
    import models
    Base.metadata.create_all(bind=engine)

@contextmanager
def get_db_session():
    """ Provide a transactional scope around a series of operations. """
    session = db_session()
    try:
        yield session
        session.commit()
    except Exception:
        session.rollback()
        raise
    finally:
        session.close()


## Base Model

In [ ]:
from sqlalchemy import Column, Integer

class BaseModel(Base):
    __abstract__ = True
    id = Column(Integer, primary_key=True, autoincrement=True)


## Example DAO

In [ ]:
class UserDAO:
    def __init__(self, session):
        self.session = session

    def add_user(self, user):
        self.session.add(user)
        
    def get_user_by_id(self, user_id):
        return self.session.query(User).get(user_id)

    def delete_user(self, user_id):
        user = self.get_user_by_id(user_id)
        if user:
            self.session.delete(user)

# Usage Example
with get_db_session() as session:
    user_dao = UserDAO(session)
    new_user = User(name="John Doe", email="john.doe@example.com")
    user_dao.add_user(new_user)


## Integrating DAO in Service Layer

In [ ]:
class UserService:
    def __init__(self, dao):
        self.dao = dao

    def create_user(self, name, email):
        user = User(name=name, email=email)
        self.dao.add_user(user)

    def get_user(self, user_id):
        return self.dao.get_user_by_id(user_id)

    def remove_user(self, user_id):
        self.dao.delete_user(user_id)


## FastAPI Web Layer
### Models

In [ ]:
from sqlalchemy import Column, String

class User(BaseModel):
    __tablename__ = "users"
    name = Column(String, nullable=False)
    email = Column(String, unique=True, nullable=False)


### Schemas

In [ ]:
from pydantic import BaseModel as PydanticBaseModel

class UserCreate(PydanticBaseModel):
    name: str
    email: str

class UserRead(PydanticBaseModel):
    id: int
    name: str
    email: str

    class Config:
        orm_mode = True


### Main Application

In [ ]:
from fastapi import FastAPI, Depends, HTTPException, status
from sqlalchemy.orm import Session
from typing import List

app = FastAPI()

def get_session():
    with get_db_session() as session:
        yield session

@app.post("/users/", response_model=UserRead, status_code=status.HTTP_201_CREATED)
def create_user(user: UserCreate, session: Session = Depends(get_session)):
    user_service = UserService(UserDAO(session))
    user_service.create_user(user.name, user.email)
    return user

@app.get("/users/{user_id}", response_model=UserRead)
def read_user(user_id: int, session: Session = Depends(get_session)):
    user_service = UserService(UserDAO(session))
    db_user = user_service.get_user(user_id)
    if db_user is None:
        raise HTTPException(status_code=404, detail="User not found")
    return db_user

@app.delete("/users/{user_id}", status_code=status.HTTP_204_NO_CONTENT)
def delete_user(user_id: int, session: Session = Depends(get_session)):
    user_service = UserService(UserDAO(session))
    user_service.remove_user(user_id)
    return {"message": "User deleted successfully"}

if __name__ == '__main__':
    import uvicorn
    uvicorn.run(app, host='0.0.0.0', port=8000)


## Conclusion

By following this architecture, you've ensured clear separation of concerns, making your database layer highly reusable and maintainable across different projects, all while leveraging FastAPI's capabilities for building modern web APIs.